In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load the data

users = pd.read_csv('takehome_users.csv', encoding='latin1')
engage = pd.read_csv('takehome_user_engagement.csv')

In [3]:
# Check the users data

users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [4]:
# Check the engagement data

engage.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [5]:
engage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [6]:
# The engagement data will be easier to work with if it's in datetime format, so I'll convert it to that

from datetime import datetime

test = []

for i in range(len(engage)):
    test.append(datetime.strptime(engage[i:(i+1)]['time_stamp'].to_string(index=False),'%Y-%m-%d %H:%M:%S'))
    
engage_time = pd.DataFrame(test,columns=['time_stamp'])
engage['time_stamp'] = engage_time
engage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null datetime64[ns]
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 MB


In [7]:
# Setup a filter to only keep users who have at least 3 logins (less than 3 can't possibly be "adopted users")
may_adopt = pd.value_counts(engage['user_id'].values).sort_index() >= 3
# Transform the filter into a dataframe to work with all indexes
may_adopt = pd.DataFrame(may_adopt)
# Somme users have 0 logins and aren't even in the engage dataframe, so I need to fill in the missing values
may_adopt = may_adopt.reindex(index = np.arange(1,12001),fill_value=False)
# Now transform the dataframe back into a filter
may_adopt = list(may_adopt[0])

users_filter = users[may_adopt]

engage_filter = users_filter['object_id']

In [8]:
# Filter out the rows of engage that can't be adopted to reduce the computation

engage_test = np.empty(len(engage))

for i in range(len(engage_test)):
    engage_test[i] = 0
    if(engage['user_id'][i] in engage_filter.values):
        engage_test[i] = 1

engage_test = engage_test.astype(bool)

# Apply the filter to the engagement data
engage_may_adopt = engage[engage_test]
# Index must be reset for this to work
engage_may_adopt = engage_may_adopt.reset_index()

engage_may_adopt.head()

,index,time_stamp,user_id,visited
0,1,2013-11-15 03:45:04,2,1
1,2,2013-11-29 03:45:04,2,1
2,3,2013-12-09 03:45:04,2,1
3,4,2013-12-25 03:45:04,2,1
4,5,2013-12-31 03:45:04,2,1


In [9]:
# Create a data frame consisting of the engagement counts, which I'll use to determine if a user is "adopted"
# and a boolean to mark a user as adopted, defaulted to false, which I'll switch if the user is "adopted"

engage_user_count = engage_may_adopt['user_id'].value_counts().sort_index()
engage_test_df = pd.DataFrame(engage_user_count)
engage_test_df = engage_test_df.rename(columns={'user_id':'engage_count'})
engage_test_df = engage_test_df.rename_axis('user_id', axis='columns')
engage_test_df['adopt'] = False

engage_test_df.head()

user_id,engage_count,adopt
2,14,False
10,284,False
20,7,False
33,18,False
42,342,False


In [10]:
# Add only the time stamps of the relevant users to the data frame

# Create an empty list to hold the lists of engagement time stamps
engage_list = []
# Create a counter to pull the time stamp from
current_stamp = 0

for i in range(len(engage_test_df)):
    # Creat a temp list to store the time stamps of the current user
    test_list = []
    # Loop over all users to capture their time stamps
    for i in range(int(engage_test_df['engage_count'][i:(i+1)].values)):
        test_list.append(engage_may_adopt['time_stamp'][current_stamp])
        # Increment time stamp counter
        current_stamp += 1
    # Append the current user's list to the overall users list
    test_list.sort()
    engage_list.append(test_list)
    
engage_test_df['time_stamp_list'] = engage_list
engage_test_df.head()

user_id,engage_count,adopt,time_stamp_list
2,14,False,"[2013-11-15 03:45:04, 2013-11-29 03:45:04, 201..."
10,284,False,"[2013-01-16 22:08:03, 2013-01-22 22:08:03, 201..."
20,7,False,"[2014-03-11 11:46:38, 2014-03-12 11:46:38, 201..."
33,18,False,"[2014-03-11 06:29:09, 2014-03-17 06:29:09, 201..."
42,342,False,"[2012-11-13 19:05:07, 2012-12-16 19:05:07, 201..."


In [11]:
# Create a function to decide if a list of time stamps contains 3 instances within any given week

from datetime import timedelta

def three_in_a_week(time_stamps):
    # Assume no 3 time stamps in a week
    week_count = False
    # Determine number of time stamps
    num_stamps = len(time_stamps)
    
    # Since there's at least 3 time stamps in each list, number of loops can be reduced by 2
    for i in range(num_stamps - 2):
        if (time_stamps[(i+2)] - time_stamps[i]) <= timedelta(days=7):
            # If time difference between time stamp 3 and time stamp 1 is within a week, set status to true and break loop
            week_count = True
            break
            
    # Return final answer
    return week_count

In [12]:
# Run all user data through function, save the results and reset user adopted flag

adopt_flag = np.empty(len(engage_test_df))
    
for i in range(len(engage_test_df)):
    adopt_flag[i] = three_in_a_week(engage_test_df['time_stamp_list'][i:(i+1)].values[0])

adopt_flag = adopt_flag == 1

adopt_users = users_filter[adopt_flag]
adopt_users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
9,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0
19,20,2014-03-06 11:46:38,Helms Mikayla,lqyvjilf@uhzdq.com,SIGNUP,1.401364e+09,0,0,58,NaN
32,33,2014-03-11 06:29:09,Araujo José,JoseMartinsAraujo@cuvox.de,GUEST_INVITE,1.401518e+09,0,0,401,79.0
41,42,2012-11-11 19:05:07,Pinto Giovanna,GiovannaCunhaPinto@cuvox.de,SIGNUP,1.401045e+09,1,0,235,NaN


In [13]:
# Let's see if we can see some trends in the adopted users

print(adopt_users['creation_source'].value_counts())
print(adopt_users['opted_in_to_mailing_list'].value_counts())
print(adopt_users['enabled_for_marketing_drip'].value_counts())

ORG_INVITE            574
GUEST_INVITE          369
SIGNUP                302
SIGNUP_GOOGLE_AUTH    239
PERSONAL_PROJECTS     172
Name: creation_source, dtype: int64
0    1227
1     429
Name: opted_in_to_mailing_list, dtype: int64
0    1399
1     257
Name: enabled_for_marketing_drip, dtype: int64


In [14]:
print(adopt_users['org_id'].value_counts())

4      17
1      16
7      16
2      15
3      14
9      14
13     14
0      12
5      12
62     12
6      11
58     11
44     10
24     10
20     10
10     10
31     10
8      10
82      9
35      9
34      9
52      9
33      9
61      9
30      8
16      8
63      8
11      8
72      8
49      8
       ..
373     1
374     1
402     1
312     1
344     1
406     1
315     1
327     1
86      1
410     1
51      1
323     1
125     1
397     1
330     1
164     1
362     1
294     1
276     1
220     1
143     1
223     1
376     1
229     1
253     1
349     1
158     1
244     1
384     1
348     1
Name: org_id, Length: 401, dtype: int64


In [15]:
print(adopt_users['invited_by_user_id'].value_counts())

10628.0    4
2354.0     4
8602.0     3
9510.0     3
2994.0     3
2776.0     3
7882.0     3
5910.0     3
1656.0     3
730.0      3
10163.0    3
4019.0     3
11267.0    3
2017.0     3
7107.0     3
3623.0     3
9726.0     3
879.0      3
2771.0     3
7358.0     3
5757.0     3
2367.0     3
3062.0     3
7816.0     2
5195.0     2
3388.0     2
6316.0     2
11297.0    2
3641.0     2
2175.0     2
          ..
8310.0     1
6529.0     1
853.0      1
979.0      1
2722.0     1
7736.0     1
7228.0     1
589.0      1
6647.0     1
11103.0    1
10456.0    1
2519.0     1
3064.0     1
553.0      1
445.0      1
4922.0     1
4877.0     1
5079.0     1
10050.0    1
6954.0     1
4603.0     1
6238.0     1
8259.0     1
9449.0     1
9162.0     1
1857.0     1
11800.0    1
7210.0     1
7012.0     1
316.0      1
Name: invited_by_user_id, Length: 768, dtype: int64


In [16]:
# I don't see anything very clear, so let's create a data frame of the non-adopted users and compare with that data

non_adopt_list = []

for i in range(len(may_adopt)):
    non_adopt_list.append(not(may_adopt[i]))
    
non_adopt_users = users[non_adopt_list]

non_adopt_users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0
5,6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1.387424e+09,0,0,197,11241.0


In [17]:
# Let's see if there are any trends in the non-adopted users

print(non_adopt_users['creation_source'].value_counts())
print(non_adopt_users['opted_in_to_mailing_list'].value_counts())
print(non_adopt_users['enabled_for_marketing_drip'].value_counts())

ORG_INVITE            3471
PERSONAL_PROJECTS     1880
SIGNUP                1669
GUEST_INVITE          1661
SIGNUP_GOOGLE_AUTH    1071
Name: creation_source, dtype: int64
0    7336
1    2416
Name: opted_in_to_mailing_list, dtype: int64
0    8308
1    1444
Name: enabled_for_marketing_drip, dtype: int64


In [18]:
# One thing that sticks out a little there is how many more personal projects are in the non-adopted users than than the
# adopted users

print(non_adopt_users['org_id'].value_counts())

0      298
1      213
2      183
3      151
4      136
6      121
5      111
9      102
7       98
10      93
8       82
14      79
12      67
17      66
18      64
11      63
16      62
13      58
20      56
28      55
23      53
15      53
25      51
21      50
30      50
26      49
40      49
24      48
27      47
32      46
      ... 
265      9
359      9
232      9
345      9
301      9
305      9
407      9
385      9
294      9
396      8
381      8
356      8
404      8
353      8
399      8
354      8
378      8
270      8
415      8
235      7
364      7
397      7
315      7
304      7
386      5
400      5
395      5
387      5
352      4
416      2
Name: org_id, Length: 417, dtype: int64


In [19]:
print(non_adopt_users['invited_by_user_id'].value_counts())

10741.0    12
1525.0     10
2527.0     10
2308.0     10
6645.0      9
7012.0      9
7716.0      9
6808.0      9
7698.0      9
10481.0     9
6493.0      8
8426.0      8
1175.0      8
3944.0      8
4118.0      8
9023.0      8
4803.0      8
1114.0      8
8068.0      8
11770.0     8
3789.0      8
9964.0      8
2922.0      8
4596.0      8
9434.0      8
4302.0      8
6370.0      8
3510.0      7
7836.0      7
6272.0      7
           ..
934.0       1
466.0       1
387.0       1
11158.0     1
1436.0      1
3620.0      1
5200.0      1
2746.0      1
8371.0      1
11848.0     1
743.0       1
2587.0      1
422.0       1
11005.0     1
8461.0      1
7979.0      1
2692.0      1
6747.0      1
4290.0      1
4241.0      1
3013.0      1
11219.0     1
9653.0      1
11972.0     1
10839.0     1
1951.0      1
2923.0      1
7413.0      1
2359.0      1
2734.0      1
Name: invited_by_user_id, Length: 2321, dtype: int64


In [20]:
# OK, well nothing's really popping out as a factor, positive or negative, so one last approach is to create a classifier
# and see what features are important to it.  I've had luck with random forests, so I'll use that.  I'll need to create a
# flag to show an adopted user.

adopt_clf_flag = []

for i in range(len(users)):
    if (users['object_id'][i] in adopt_users['object_id'].values):
        adopt_clf_flag.append(1)
    else:
        adopt_clf_flag.append(0)

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

users_test = users.drop(['object_id','name','email','creation_time','last_session_creation_time','invited_by_user_id'], axis = 1)
users_test = pd.get_dummies(users_test, columns = ['creation_source'])

X_train, X_test, y_train, y_test = train_test_split(users_test, adopt_clf_flag, stratify = adopt_clf_flag)

# I'm just picking some hyper-parameters for time's sake
clf = RandomForestClassifier(max_features = None, n_estimators = 100)

clf.fit(X_train, y_train)
print('Classifier score :',clf.score(X_test, y_test))

feat_imp = pd.DataFrame({'Features': users_test.columns, 'Estimated Coefficients': np.reshape(clf.feature_importances_,8)})
feat_imp = feat_imp.reindex(feat_imp['Estimated Coefficients'].sort_values(ascending=False).index)
print('\nFeature Importances :')
print(feat_imp)

Classifier score : 0.8186666666666667

Feature Importances :
   Estimated Coefficients                            Features
2                0.715587                              org_id
0                0.091676            opted_in_to_mailing_list
1                0.069204          enabled_for_marketing_drip
6                0.033891              creation_source_SIGNUP
3                0.027980        creation_source_GUEST_INVITE
4                0.027416          creation_source_ORG_INVITE
7                0.024024  creation_source_SIGNUP_GOOGLE_AUTH
5                0.010223   creation_source_PERSONAL_PROJECTS


That classifier score is not bad, and the feature importances suggest the only feature that really matters is which or the user belongs to.  However, looking back at the lists of the users' orgs, the single digit orgs appear a lot in both the adopted and non-adopted users, so studying this will require more time.